<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

In [29]:
import yaml

# Set `PATH` to include the directory containing TFX CLI.
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [30]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.21.2


# Continuous training with TFX and Cloud AI Platform

This lab walks you through a TFX pipeline that uses **Cloud Dataflow** and **Cloud AI Platform Training** as execution runtimes. The pipeline implements a typical TFX workflow as depicted on the below diagram:



## Understanding the pipeline design
The pipeline source code can be found in the `pipeline` folder.

In [2]:
!ls -la pipeline

total 68
drwxr-xr-x 3 jupyter jupyter 4096 Mar 18 22:33 .
drwxr-xr-x 5 jupyter jupyter 4096 Mar 18 22:33 ..
-rw-r--r-- 1 jupyter jupyter 2150 Mar 18 22:30 beam_runner.py
-rw-r--r-- 1 jupyter jupyter 1325 Mar 18 22:30 config.py
-rw-r--r-- 1 jupyter jupyter 1222 Mar 18 22:30 features.py
drwxr-xr-x 2 jupyter jupyter 4096 Mar 18 22:32 .ipynb_checkpoints
-rw-r--r-- 1 jupyter jupyter 6565 Mar 18 22:30 model.py
-rw-r--r-- 1 jupyter jupyter 7251 Mar 18 22:30 pipeline.py
-rw-r--r-- 1 jupyter jupyter 2036 Mar 18 22:30 preprocessing.py
-rw-r--r-- 1 jupyter jupyter 3073 Mar 18 22:33 runner.py
-rw-r--r-- 1 jupyter jupyter 7576 Mar 18 22:30 test.py
-rw-r--r-- 1 jupyter jupyter 8272 Mar 18 22:30 transform_train.py


The `config.py` module configures the default values for the environment specific settings and the default values for the pipeline runtime parameters. 
The default values can be overwritten at compile time by providing the updated values in a set of environment variables.

In [3]:
!tail -n 15 pipeline/config.py

class Config:
    """Sets configuration vars."""
    
    PIPELINE_NAME=os.getenv("PIPELINE_NAME", "covertype_continuous_training")
    MODEL_NAME=os.getenv("MODEL_NAME", "covertype_classifier")
    PROJECT_ID=os.getenv("PROJECT_ID", "mlops-workshop")
    GCP_REGION=os.getenv("GCP_REGION", "us-central1")
    TFX_IMAGE=os.getenv("KUBEFLOW_TFX_IMAGE", "tensorflow/tfx:0.21.2")
    DATA_ROOT_URI=os.getenv("DATA_ROOT_URI", "gs://workshop-datasets/covertype/small")
    ARTIFACT_STORE_URI=os.getenv("ARTIFACT_STORE_URI", "gs://mlops-workshop-artifact-store")
    RUNTIME_VERSION=os.getenv("RUNTIME_VERSION", "2.1")
    PYTHON_VERSION=os.getenv("PYTHON_VERSION", "3.7")
    
    
    

The `pipeline.py` module contains the TFX DSL defining the workflow implemented by the pipeline.

The `transform_train.py` module implements data preprocessing and training logic for the `Transform` and `Train` components.

The `runner.py` module configures and executes `KubeflowDagRunner`. At compile time, the `KubeflowDagRunner.run()` method conversts the TFX DSL into the pipeline package in the [argo](https://argoproj.github.io/argo/) format.

The TFX components defined in the pipeline execute in a runtime provided by a custom docker image, which is a derivate of the base TFX image from Docker Hub - tensorflow/tfx:0.21.x. The custom image updates the base image with the latest TFX libraries and embeds the Python modules for `Transform` and `Train` components and the `schema` file defining input data. 



In [31]:
!cat Dockerfile

FROM tensorflow/tfx:0.21.2
WORKDIR /pipeline
COPY pipeline/* ./
COPY schema/schema.pbtxt ./schema/
ENV PYTHONPATH="/pipeline:${PYTHONPATH}"

## Building and deploying the pipeline

You will use TFX CLI to compile and deploy the pipeline. As explained in the previous section, the environment specific settings can be provided through a set of environment variables and embedded into the pipeline package at compile time.

### Configure environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `GCP_REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.
- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [32]:
GCP_REGION = 'us-central1'
ENDPOINT = '30eb2309b051656-dot-us-central2.pipelines.googleusercontent.com'
ARTIFACT_STORE_URI = 'gs://hostedkfp-default-9t1yc9uxwn'

In [33]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

DATA_ROOT_URI = 'gs://workshop-datasets/covertype/small'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/covertype-tfx-image'.format(PROJECT_ID)
PIPELINE_NAME = 'tfx_covertype_continuous_training'
MODEL_NAME = 'tfx_covertype_classifier'
RUNTIME_VERSION = '2.1'
PYTHON_VERSION = '3.7'

In [34]:
%env PROJECT_ID={PROJECT_ID}
%env KUBEFLOW_TFX_IMAGE={CUSTOM_TFX_IMAGE}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env GCP_REGION={GCP_REGION}
%env MODEL_NAME={MODEL_NAME}
%env PIPELINE_NAME={PIPELINE_NAME}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERIONS={PYTHON_VERSION}

env: PROJECT_ID=caip-demo
env: KUBEFLOW_TFX_IMAGE=gcr.io/caip-demo/covertype-tfx-image
env: ARTIFACT_STORE_URI=gs://hostedkfp-default-9t1yc9uxwn
env: DATA_ROOT_URI=gs://workshop-datasets/covertype/small
env: GCP_REGION=us-central1
env: MODEL_NAME=tfx_covertype_classifier
env: PIPELINE_NAME=tfx_covertype_continuous_training
env: RUNTIME_VERSION=2.1
env: PYTHON_VERIONS=3.7


### Compile the pipeline

You can build and upload the pipeline to the AI Platform Pipelines instance in one step, using the `tfx pipeline create` command. The `tfx pipeline create` goes through the following steps:
- (Optional) Builds the custom image to host your components, 
- Compiles the pipeline DSL into a pipeline package 
- Uploads the pipeline package to the instance.

As you debug the pipeline DSL, you may prefer to first use the `tfx pipeline compile` command, which only executes the compilation step. After the DSL compiles successfully you can use `tfx pipeline create` to go through all steps.

In [35]:
!tfx pipeline compile --engine kubeflow --pipeline_path pipeline/runner.py

CLI
Compiling pipeline
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/mlops-labs/workshops/tfx-caip-tf21/lab-02-tfx-pipeline/tfx_covertype_continuous_training.tar.gz


### Deploy the pipeline package to AI Platform Pipelines

After the pipeline code compiles without any errors you can use the `tfx pipeline create` command to perform the full build and deploy the pipeline. 

In [36]:
!tfx pipeline create  \
--pipeline_path=pipeline/runner.py \
--endpoint={ENDPOINT} \
--build_target_image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/mlops-labs/workshops/tfx-caip-tf21/lab-02-tfx-pipeline/tfx_covertype_continuous_training.tar.gz
{'created_at': datetime.datetime(2020, 3, 19, 0, 33, 20, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2020, 3, 19, 0, 33, 20, tzinfo=tzlocal()),
                     'id': 'af0c4ac3-b262-41ed-aa17-fe3649975d21',
           

If you need to redeploy the pipeline you can first delete the previous version using `tfx pipeline delete` or you can update the pipeline in-place using `tfx pipeline update`.

To delete the pipeline.

In [13]:
!tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

CLI
Deleting pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Pipeline "tfx_covertype_continuous_training" does not exist.


### Create and monitor a pipeline run
After the pipeline has been deployed, you can trigger and monitor pipeline runs using TFX CLI or KFP UI.

To submit the pipeline run using TFX CLI:

In [37]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: tfx_covertype_continuous_training
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: tfx_covertype_continuous_training
+-----------------------------------+--------------------------------------+----------+---------------------------+
| pipeline_name                     | run_id                               | status   | created_at                |
+===================================+======================================+==========+===========================+
| tfx_covertype_continuous_training | 42f4341a-85b3-4961-bf01-99f7eaf8a80e |          | 2020-03-19T00:33:33+00:00 |
+-----------------------------------+--------------------------------------+----------+---------------------------+


To list all active runs of the pipeline:

In [ ]:
!tfx run list --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

To retrieve the status of a given run:

In [ ]:
RUN_ID='[YOUR RUN ID]'

!tfx run status --pipeline_name {PIPELINE_NAME} --run_id {RUN_ID} --endpoint {ENDPOINT}